In [ ]:
import time
import threading
from IPython.display import display
import ipywidgets as widgets

# function to be executed by interactive() method
def f(a, b): 
    return a+b 

"""
Idee: Use Asynchronous Widgets together with interactive() method

Tatsächlich sind wir nicht am Output der Funktion f()
interessiert. Wichtig für uns ist, dass die Widget instance
'w' die aktuellen Slider Werte beinhält und diese auch abgerufen
werden können. Über 'w.kwargs' können die Werte der einzelnen 
Slider abgefragt werden und über 'w.result' kann der return value
der Funktion f() abgerufen werden.

Da die Funktion 'work()' auf thread 2 ständig 'w.kwargs' und 'w.result'
abfragt, welche an den Slider auf thread 1 gebunden sind, werden Änderungen
am Slider in thread 1 direkt an 'work()' auf thread 2 übergeben.

Angewandt auf unser Problem könnte dies so aussehen:

1. Anstatt 'work()' callen wir 'change_grid_layout_time()'
2. Dieses fragt die Werte eines oder mehrerer Slider(s) ab
   und übergibt diese an 'solve_navier_stokes()'
   
   PROBLEM: Das Loop befindet sich innerhalb der Funktion
   'solve_navier_stokes()', die Werte werden aber von der
   äußeren Funktion 'change_grid_layout_time()' abgefragt.
   Dies läuft also wieder auf eine ähnliche Situation hinaus
   wie bisher. Man muss die Werte-Abfrage irgenwie in das Loop
   in 'solve_navier_stokes()' hineinpacken.
   
   LÖSUNG: Eigentlich ist die Funktion 'change_grid_layout_time()'
   redundant, da diese nur die Werte der Slider abfragt und dann
   erst recht 'solve_navier_stokes()' aufruft. Dies ließe sich alles
   in einer einzigen Funktion kombinieren.

"""

# w stores data bound to slider
w = widgets.interactive(f, a=10, b=20)

# this is the main function running on a separate thread
# input to this function is the data bound to the slider widget
def work(w):
    for i in range(100):
        time.sleep(0.5)
        print(w.kwargs, w.result)

# create thread for running the main function
thread = threading.Thread(target=work, args=(w,))

# To actually display the widgets, use IPython’s display function.
display(w)

# start thread for running the main function
thread.start()

In [ ]:
import time
import threading
from IPython.display import display
import ipywidgets as widgets

# function to be executed by interactive() method
def f(a, b): 
    return a+b 

# w stores data bound to slider
w = widgets.interactive(f, a=10, b=20)

def work():
    a_local = (w.kwargs)["a"]
    for i in range(100):
        if (w.kwargs)["a"] != a_local:
            a_local = (w.kwargs)["a"]
            print("Slider value has changed")
            # das führt zu race conditions, bei denen dieses "if" statement
            # noch nicht ausgeführt wird aber unten bei print(w.kwargs) dann
            # schon ein neuerer Wert verwendet wird. 
            
            # dies wird zu Problem, wenn eigentlich die "a" Matrix neu assembliert
            # werden sollte bevor dann der "convection" part kommt, aber dies eventuell
            # nicht geschieht
            
            # Workaround ---> immer assemblieren, nicht erst wenn Slider Wert geändert wird
            # ODER: einen timestep dann auch im if-statement ausführen ---> ändert aber glaub ich
            # nichts. Wenn der Slider währenddessen weiterbewegt wird, passiert das wieder und wieder
            
            # Workaround ---> Slider nicht "continuous" machen
            
            # GIBTS die möglichekeit, w.kwargs nur 1x per loop iteration abzurufen und zu speichern
            # und für alles zu verwenden und trotzdem auf geänderte werte abzufragen?
            
            # Ich glaube so wie es jetzt ist passts ---> Jep ziemlich sicher wie ein kurzer Test zeigt
            # und noch sicherer wenn man sich die Logik durchdenkt
                           
        time.sleep(0.5)
        #print(w.kwargs)
        print(a_local)
        
# create thread for running the main function
thread = threading.Thread(target=work)

# To actually display the widgets, use IPython’s display function.
display(w)

# start thread for running the main function
thread.start()